In [1]:
from pathlib import Path
from src.inspector_git.linker.models import GitProject
from src.inspector_git.linker.transformers import GitProjectTransformer
from src.inspector_git.reader.iglog.readers.ig_log_reader import IGLogReader
from src.run_blame import check_blame
import sys
# sys.setrecursionlimit(16336)

def build_and_check_project(iglog_path: str, repo_path: str, compute_annotated_lines :bool) -> GitProject:
    iglog_file = Path(iglog_path)

    with open(iglog_file, "r", encoding="utf-8") as f:
        git_log_dto = IGLogReader().read(f)

    transformer = GitProjectTransformer(
        git_log_dto,
        name=iglog_file.stem,
        compute_annotated_lines=compute_annotated_lines,
    )
    project = transformer.transform()

    if compute_annotated_lines:
        check_blame(project, repo_path)

    return project


In [2]:
project = build_and_check_project(
    "../../test-input/inspector-git/TestInspectorGitRepo.iglog",
    "/home/vortex/Work/BachelorThesis/voyager-target/zeppelin/zeppelin",
    False
)


2025-09-20 23:48:17,418 [INFO] src.inspector_git.linker.transformers: Creating GIT project TestInspectorGitRepo
2025-09-20 23:48:17,419 [INFO] src.inspector_git.linker.transformers: Creating commit 1 / 8 (12%)
2025-09-20 23:48:17,423 [INFO] src.inspector_git.linker.transformers: Creating commit 2 / 8 (25%)
2025-09-20 23:48:17,424 [INFO] src.inspector_git.linker.transformers: Creating commit 3 / 8 (37%)
2025-09-20 23:48:17,425 [INFO] src.inspector_git.linker.transformers: Creating commit 4 / 8 (50%)
2025-09-20 23:48:17,425 [INFO] src.inspector_git.linker.transformers: Creating commit 5 / 8 (62%)
2025-09-20 23:48:17,426 [INFO] src.inspector_git.linker.transformers: Creating commit 6 / 8 (75%)
2025-09-20 23:48:17,427 [INFO] src.inspector_git.linker.transformers: Creating commit 7 / 8 (87%)
2025-09-20 23:48:17,427 [INFO] src.inspector_git.linker.transformers: Creating commit 8 / 8 (100%)
2025-09-20 23:48:17,428 [INFO] src.inspector_git.linker.transformers: Done creating GIT project TestIns

In [3]:
import pickle
# import sys
# sys.setrecursionlimit(100000)

with open("project.pkl", "wb") as f:
    pickle.dump(project, f)

print("GitProject saved to project.pkl")

GitProject saved to project.pkl


In [4]:
# --- Load from pickle ---
with open("project.pkl", "rb") as f:
    loaded_project = pickle.load(f)

In [1]:
from typing import List, Dict, Any
from pydantic import BaseModel
import pickle

# --- Models ---
class Student(BaseModel):
    id: int
    first_name: str
    last_name: str


class Classroom(BaseModel):
    id: str
    students: List[Student]

    # Custom serialization (to dict-like form)
    def to_dict(self) -> Dict[str, Any]:
        return {
            "id": self.id,
            "student_ids": [s.id for s in self.students],
        }

    # Pickling support using __reduce__
    def __reduce__(self):
        # Instead of storing Student objects, store only IDs
        state = (self.id, [s.id for s in self.students])
        return (self._rebuild, state)

    @classmethod
    def _rebuild(cls, id: str, student_ids: List[int]):
        # Rebuild with empty students list, keep IDs in a temp attr
        obj = cls(id=id, students=[])
        obj._student_ids = student_ids
        return obj

    def attach_students(self, student_lookup: Dict[int, Student]):
        """Rebuild students list from a lookup dict of all students."""
        self.students = [student_lookup[sid] for sid in getattr(self, "_student_ids", [])]
        if hasattr(self, "_student_ids"):
            del self._student_ids

In [4]:
# --- Example usage ---
if __name__ == "__main__":
    # Create students
    s1 = Student(id=1, first_name="Alice", last_name="Smith")
    s2 = Student(id=2, first_name="Bob", last_name="Jones")

    # Create classroom
    c1 = Classroom(id="room101", students=[s1, s2])

    # Serialize (pickle)
    data = pickle.dumps(c1)

    # Deserialize
    restored = pickle.loads(data)
    print(restored._student_ids)

    # # Reattach students from global registry
    # student_registry = {s.id: s for s in [s1, s2]}
    # restored.attach_students(student_registry)
    #
    # print(restored)
    # print(restored.students)

[1, 2]


In [5]:
c1._ceva_nou = 1
